In [11]:
import random
import requests_html
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urlparse
from requests_html import HTMLSession


In [32]:
# Function to grab random user agent
def get_user_agent():
    uastrings = ["Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.111 Safari/537.36",\
                "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/28.0.1500.72 Safari/537.36",\
                "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10) AppleWebKit/600.1.25 (KHTML, like Gecko) Version/8.0 Safari/600.1.25",\
                "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:33.0) Gecko/20100101 Firefox/33.0",\
                "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.111 Safari/537.36",\
                "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.111 Safari/537.36",\
                "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit/600.1.17 (KHTML, like Gecko) Version/7.1 Safari/537.85.10",\
                "Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko",\
                "Mozilla/5.0 (Windows NT 6.3; WOW64; rv:33.0) Gecko/20100101 Firefox/33.0",\
                "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.104 Safari/537.36"\
                "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:15.0) Gecko/20100101 Firefox/15.0.1"\
                "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.111 Safari/537.36"\
                "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9"\
                "Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36"\
                "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"\
                ]
 
    return random.choice(uastrings)

# Function to try faking a random user agent and return parsed DOM if successful
def parse_url(url):
    # Get the user agent
    headers = {'User-Agent': get_user_agent()}
    content = None
    # Try to get the content with faked user agent
    try:
        response = requests.get(url, headers = headers)
        ct = response.headers['Content-Type'].lower().strip()
        # If the content is HTML, parse it
        if 'text/html' in ct:
            # Dumping the content from response to a variable
            content = response.content
            # Parsing the content using BeautifulSoup
            soup = BeautifulSoup(content, "html.parser")
        # If the content is not HTML, do not parse
        else:
            content = response.content
            soup = None
    # If error, print error
    except Exception as e:
        print("Error:", str(e))
    # Return the variables
    return soup

# Strip grab internal links from the content
def parse_internal_links(soup, current_page):
    return [a['href'].lower().strip() for a in soup.find_all('a', href = True) if urlparse(a['href']).netloc == urlparse(current_page).netloc]


In [33]:
# URL to scrape
url = "https://www.yelp.com/search?find_desc=Restaurants&find_loc=Kansas+City%2C+KS&ns=1"
#url = "https://www.zomato.com/kansas-city/restaurants?rating_range=4.0-5.0"
soup = parse_url(url)

#soup

In [34]:
## Need to find the right class to grab the data
top_spots = soup.find_all("div", attrs = {"class": "arrange-unit__09f24__rqHTg arrange-unit-fill__09f24__CUubG border-color--default__09f24__NPAKY"})
print(top_spots)
list_tr = top_spots[0].find_all("div", attrs = {"class": "col-s-8 col-l-1by3"})

[<div class="arrange-unit__09f24__rqHTg arrange-unit-fill__09f24__CUubG border-color--default__09f24__NPAKY"><div class="search-suggest__09f24__Yd8a9"><div class=""><form action="/search" class="find-near-form__09f24__qHR1I" id="header_find_form" method="get" role="search"><div class="undefined arrange__09f24__LDfbs border-color--default__09f24__NPAKY"><div class="arrange-unit__09f24__rqHTg arrange-unit-fill__09f24__CUubG border-color--default__09f24__NPAKY"><div class="find-near-arrange__09f24__czL3z find-near-arrange--height__09f24__uFYD6 arrange__09f24__LDfbs layout-equal__09f24__odeM0 border-color--default__09f24__NPAKY"><div class="find-near-arrange-unit__09f24__hvHOu arrange-unit__09f24__rqHTg border-color--default__09f24__NPAKY"><label class="label__09f24__2EYit pseudo-input__09f24__nfxxT pseudo-input--find-near__09f24__o_Lff pseudo-input--find__09f24__dWbac pseudo-input__09f24__bK86t" for="search_description"><span class="pseudo-input-text__09f24__PW2ow display--inline__09f24__

In [28]:
restaurants = []
for tr in list_tr:
    dataframe = {}
    dataframe['name'] = tr.find("div", attrs = {"class": "col-s-16 col-l-15"}).find("a").text

In [29]:
restaurants = []
for tr in list_tr:
    dataframe = {}
    dataframe["name"] = (tr.find("div",attrs={"class": "res_title zblack bold nowrap"})).text.replace('\n', ' ')
    dataframe["address"] = (tr.find("div",attrs={"class": "nowrap grey-text fontsize5 ttupper"})).text.replace('\n', ' ')
    dataframe["cuisine_type"] = (tr.find("div",attrs={"class":"nowrap grey-text"})).text.replace('\n', ' ')
    restaurants.append(dataframe)
restaurants

[]

In [30]:
import pandas
df = pandas.DataFrame(restaurants)
df.to_csv("zomato_restaurants.csv", index = False)